In [4]:
import numpy as np
import pandas as pd
import datetime as dt

from datetime import datetime, date
from pandas.tseries.offsets import CustomBusinessDay

In [26]:
# abre uma amostra pequena para testes
amo = pd.read_excel('../../../data/economatica_0/amostra.xlsx',
                            header=0,
                            index_col = 0,
                            na_values='-')

In [27]:
amo

,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11
Data,,,,,,,,
1986-01-02,12.910000,18.220000,6.980000,3.420000,44.800000,24.000000,11.000000,72.160000
1986-01-03,13.216527,18.170662,7.326799,3.895877,45.171675,24.031133,10.533993,71.806419
1986-01-06,13.387442,18.362621,7.722260,4.281798,45.575835,24.201763,10.295437,72.113842
1986-01-07,13.644684,18.220068,8.148950,3.812366,45.757397,24.035626,10.104700,71.757336
1986-01-08,13.879209,18.068734,8.418237,4.157382,45.427298,24.496748,9.973044,71.989071
1986-01-09,14.053341,17.633781,8.680760,4.107007,45.629326,24.355712,9.641821,72.408921
1986-01-10,13.937932,17.638058,8.594629,4.154174,45.403103,23.936260,9.601688,72.030881
1986-01-13,13.956433,17.933903,8.330057,3.751991,45.534022,24.410497,10.057291,72.314981
1986-01-14,14.172767,18.087308,8.529756,3.591960,45.444564,24.733140,10.118143,72.603763


In [30]:
ind = amo.index # retorna um DatetimeIndex
ind.get_values() # retorna um np.array de datetime64. Similar a ind.values
type(ind.get_values()) # dá um numpy.ndarray

DatetimeIndex(['1986-01-02', '1986-01-03', '1986-01-06', '1986-01-07',
               '1986-01-08', '1986-01-09', '1986-01-10', '1986-01-13',
               '1986-01-14', '1986-01-15', '1986-01-16', '1986-01-17',
               '1986-01-20', '1986-01-21', '1986-01-22', '1986-01-23',
               '1986-01-24', '1986-01-27', '1986-01-28', '1986-01-29',
               '1986-01-30', '1986-01-31'],
              dtype='datetime64[ns]', name='Data', freq=None)

In [42]:
# Uma coisa que eu queria era criar um calendar a partir de uma lista de datas, mas aparentemente isso é impossível pois
# calendar é infinito e lista de datas é finito. O que dá pra fazer é inferir o calendário e tem uma função legal que faz isso
# https://stackoverflow.com/questions/54023828/create-calendar-with-pandas-dateoffset-from-list-of-dates
# na verdade a função foi feita por alguém do stackoverflow

def infer_calendar(dates):
    """
    Infer a calendar as pandas DateOffset from a list of dates.
    Parameters
    ----------
    dates : array-like (1-dimensional) or pd.DatetimeIndex
        The dates you want to build a calendar from
    Returns
    -------
    calendar : pd.DateOffset (CustomBusinessDay)
    """
    dates = pd.DatetimeIndex(dates)

    traded_weekdays = []
    holidays = []

    days_of_the_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    for day, day_str in enumerate(days_of_the_week):

        weekday_mask = (dates.dayofweek == day)

        # keep only days of the week that are present
        if not weekday_mask.any():
            continue
        traded_weekdays.append(day_str)

        # look for holidays
        used_weekdays = dates[weekday_mask].normalize()
        all_weekdays = pd.date_range(dates.min(), dates.max(),
                                     freq=CustomBusinessDay(weekmask=day_str)
                                     ).normalize()
        _holidays = all_weekdays.difference(used_weekdays)
        _holidays = [timestamp.date() for timestamp in _holidays]
        holidays.extend(_holidays)

    traded_weekdays = ' '.join(traded_weekdays)
    return CustomBusinessDay(weekmask=traded_weekdays, holidays=holidays)

In [44]:
# Testa a função infer_calendar
dates = amo.index
calendar = infer_calendar(dates)

In [45]:
    print("dates:", dates)
    print("calendar:", pd.date_range(dates[0], dates[-1], freq=calendar) )

dates: DatetimeIndex(['1986-01-02', '1986-01-03', '1986-01-06', '1986-01-07',
               '1986-01-08', '1986-01-09', '1986-01-10', '1986-01-13',
               '1986-01-14', '1986-01-15', '1986-01-16', '1986-01-17',
               '1986-01-20', '1986-01-21', '1986-01-22', '1986-01-23',
               '1986-01-24', '1986-01-27', '1986-01-28', '1986-01-29',
               '1986-01-30', '1986-01-31'],
              dtype='datetime64[ns]', name='Data', freq=None)
calendar: DatetimeIndex(['1986-01-02', '1986-01-03', '1986-01-06', '1986-01-07',
               '1986-01-08', '1986-01-09', '1986-01-10', '1986-01-13',
               '1986-01-14', '1986-01-15', '1986-01-16', '1986-01-17',
               '1986-01-20', '1986-01-21', '1986-01-22', '1986-01-23',
               '1986-01-24', '1986-01-27', '1986-01-28', '1986-01-29',
               '1986-01-30', '1986-01-31'],
              dtype='datetime64[ns]', freq='C')


In [12]:
# pra ver quais variáveis tem no diretório
dir()
# uma maneira interessante de deletar variáveis para liberar memória
# https://stackoverflow.com/questions/22934204/how-to-clear-variables-in-ipython
%reset_selective -f "cp_ati"

In [23]:
# closing price monthly
cp_mon = pd.read_excel('../../../data/economatica_0/close_monthly.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
cp_mon.drop(["Jan-86", "Feb-86"], inplace=True)
#cp_mon = org_columns(cp_mon)


In [13]:
cpd_ati = pd.read_excel('../../../data/economatica_1/fechamento_diario_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
cpd_can = pd.read_excel('../../../data/economatica_1/fechamento_diario_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
cpd = pd.concat([cpd_ati,cpd_can],axis=1,sort=False)
cpd.columns = cpd.columns.str.extract('(.*)$')[0]
cpd.columns.name = "ticker"

In [21]:
cpd

ticker,QVQP3B,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,ALEF3B,...,WHMT3,WISA3,WISA4,WWOW3,ILMD3,ILMD4,ESTC4,ESTC11,OPZI3B,ZIVI4
Data,,,,,,,,,,,,,,,,,,,,,
1986-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.465951e-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.480085e-10,NaN,NaN,NaN,NaN,2.916550e-08,NaN,NaN,NaN,NaN
1986-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.480085e-10,NaN,NaN,NaN,NaN,2.858219e-08,NaN,NaN,NaN,NaN
1986-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.480085e-10,NaN,NaN,NaN,NaN,2.858219e-08,NaN,NaN,NaN,NaN
1986-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.773411e-10,NaN,NaN,NaN,NaN,2.858219e-08,NaN,NaN,NaN,NaN
1986-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.183281e-10,NaN,NaN,NaN,NaN,2.858219e-08,NaN,NaN,NaN,NaN
1986-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.773411e-10,NaN,NaN,NaN,NaN,2.858219e-08,NaN,NaN,NaN,NaN
1986-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.773411e-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.490742e-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
# notar como o numpy também concorre em funções para timeseries
# o busday_count aparentemente apenas exclui fins de semana
# start = dt.date(2018,12,20)
# end = dt.date(2018,12,28)
# Counts the number of valid days between begindates and enddates, not including the day of enddates.
# As datas de entrada da função são datetime64 (do numpy), so named because “datetime” is already taken by the datetime library
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.busday_count.html
# tem o parametro holidays, que é complemento do weekmask
# days = np.busday_count(start,end)
# esse resultado dá days=6, [20(qui) 21(sex) 24(seg) 25(ter) 26(qua) 27(qui)] mas na verdade deveria dar 5 pois 25 é natal
# days
# Deu 6 mas deveria dar 5 pois natal é feriado.
# Todas as funções do numpy que dão suporte a datas estão em
# https://docs.scipy.org/doc/numpy/reference/routines.datetime.html

# Uma referência completa sobre dates 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

In [3]:
# Na função busday_count() temos duas maneiras de manejar os dias úteis
# A primeira seria setando tanto os parametros weekmask como holidays
# Outra maneira é setando um busday calendar
# Na verdade um busdaycalendar nada mais é do que a junção dos parâmetros weekmask e holidays
# Eu tentei usar a função genfromtxt e recebi um monte de NaN, eis a explicação

# By default, np.genfromtxt uses dtype=float: that's why you string columns are converted to NaNs because,
# after all, they're Not A Number...
# https://stackoverflow.com/questions/12319969/how-to-use-numpy-genfromtxt-when-first-column-is-string-and-the-remaining-column
# np.genfromtxt('feriados_nacionais.csv', delimiter=',', dtype=None)

# Bytes literals are always prefixed with 'b' or 'B'; they produce an instance of the bytes type instead of the str type.
# They may only contain ASCII characters; bytes with a numeric value of 128 or greater must be expressed with escapes.
# https://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal
# df = np.genfromtxt('feriados_nacionais.csv', delimiter=',', dtype=None)

# O melhor mesmo é usar string como dtype. Esse caso ele lê o header como se fosse dado
#df = np.genfromtxt('feriados_nacionais.csv', delimiter=',', dtype=str)
#type(df) >>> <class 'numpy.ndarray'>

# Não deixar de checar todos os parâmetros que a função genfromtxt possui
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html

# Esse é uma solução para ler só a primeira coluna, que são os feriados que interessam. Mas é melhor usar o parâmetro usecolumn
#df = np.genfromtxt('feriados_nacionais.csv', delimiter=',', dtype=str, skip_header=1)
#df = df[:,0]

# usecols Which columns to read, with 0 being the first. For example, usecols = (1, 4, 5) will extract the 2nd, 5th and 6th columns.
# tem que passar uma tupla para usecols
# os holidays vem em formato %d/%m/%Y
holidays = np.genfromtxt('feriados_nacionais.csv', delimiter=',', dtype=str, skip_header=1, usecols=(0))
#holidays = [x.replace('/','-') for x in holidays]

# Para transformar em strings, um cabra estava com o mesmo problema que o meu descrito aqui. Resolve usando comprehensions
# https://stackoverflow.com/questions/30487870/python-converting-strings-in-an-array-to-dates
# esse trecho foi feito pois eu acreditava que busdaycalendar necessitava um array de dates, mas ele precisa array de strings
# para o busdaycalendar tinhamos data com mascara %d/%m/%Y e precisávamos passar parâmetro com máscara %Y-%m-%d
# https://www.programiz.com/python-programming/datetime/strftime
# holidays = [datetime.strptime(date, "%d/%m/%Y") for date in holidays]
# holidays = [date.strftime("%Y-%m-%d") for date in holidays]

# Uma solução mais elegante e que só ocupa uma linha
holidays = [datetime.strptime(date, "%d/%m/%Y").strftime("%Y-%m-%d") for date in holidays]


# Agora é a vez do busdaycalendar
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.busdaycalendar.html
# busdaycalendar()
    # weekmask : str or array_like of bool, optional
    # holidays : array_like of datetime64[D], optional
# o formato do holidays é %Y-%m-%d
ANBIMA_Calendar = np.busdaycalendar(
    holidays=holidays, weekmask='1111100')

In [4]:
# refazendo o experimento de duas células atrás que tinha dado 6 pois não contou o natal como feriado
start = dt.date(2018,12,20)
end = dt.date(2018,12,28)

days = np.busday_count(start,end,busdaycal=ANBIMA_Calendar)
# agora o resultado dá 5 certinho
days
# Deu 5 e contou natal como feriado!

5

In [6]:
# teste 1 comparando com algoritmo do R
start = dt.date(2005,1,1)
end = dt.date(2020,12,31)

days = np.busday_count(start,end,busdaycal=ANBIMA_Calendar)
# agora o resultado dá 5 certinho
days

4016

In [ ]:
holidays = np.genfromtxt('feriados_nacionais.csv', delimiter=',', dtype=str, skip_header=1, usecols=(0))

In [8]:
# essa função read_csv é do pandas, então é natural que ela retorne um objeto pandas e que provavelmente 
# não se comunica com numpy
# >>> type(df) >>> <class 'pandas.core.frame.DataFrame'>
#df = pd.read_csv("feriados_nacionais.csv")
#df = pd.to_datetime(df["Data"])


#df = pd.read_csv("feriados_nacionais.csv",usecols=[0])
# Engraçado como a função abaixo não funciona com pd.to_datetime(df) apesar de df só ter uma coluna
# Isso acontece porque df é um Dataframe e df["Data"] é uma series
#df = pd.to_datetime(df["Data"])

# squeeze : bool, default False If the parsed data only contains one column then return a Series.
df = pd.read_csv("feriados_nacionais.csv",usecols=[0], squeeze=True)
df = pd.to_datetime(df)

DiasUteisBancario = CustomBusinessDay(holidays=df)
df

0     2001-01-01
1     2001-02-26
2     2001-02-27
3     2001-04-13
4     2001-04-21
5     2001-01-05
6     2001-06-14
7     2001-07-09
8     2001-12-10
9     2001-02-11
10    2001-11-15
11    2001-12-25
12    2002-01-01
13    2002-11-02
14    2002-12-02
15    2002-03-29
16    2002-04-21
17    2002-01-05
18    2002-05-30
19    2002-07-09
20    2002-12-10
21    2002-02-11
22    2002-11-15
23    2002-12-25
24    2003-01-01
25    2003-03-03
26    2003-04-03
27    2003-04-18
28    2003-04-21
29    2003-01-05
         ...    
906   2076-06-18
907   2076-07-09
908   2076-12-10
909   2076-02-11
910   2076-11-15
911   2076-12-25
912   2077-01-01
913   2077-02-22
914   2077-02-23
915   2077-09-04
916   2077-04-21
917   2077-01-05
918   2077-10-06
919   2077-07-09
920   2077-12-10
921   2077-02-11
922   2077-11-15
923   2077-12-25
924   2078-01-01
925   2078-02-14
926   2078-02-15
927   2078-01-04
928   2078-04-21
929   2078-01-05
930   2078-02-06
931   2078-07-09
932   2078-12-10
933   2078-02-

In [22]:
hoje = dt.datetime(2018,12,21)
hoje+5*DiasUteisBancario

Timestamp('2018-12-31 00:00:00')

In [17]:
hoje

datetime.datetime(2018, 12, 21, 0, 0)